# Refresh questions

- @author Gong Zequn (zequn.gong@u.nus.edu)
- @create 2021.07.17
- @modify 2021.07.22
- @author Shashank Acharya (acharya.s@u.nus.edu)
- @modify 2022.03.22

# Procedure

We cannot just delete the old questions and then add new questions, since we are using `question_id` in the answer table to indicate which question does this answer belong to.

So, we will do the following to insert new questions:
- start by setting the `enable` column for all questions to `0`.
- for each question, insert it into database (the default `enable` will be `1`)
- if a question is in the list that is already in the database, set the `enable` of the row to `1`


The SQL should be like:

```sql
BEGIN;
UPDATE `question` SET `enable`=0;
INSERT INTO `question` (`word`) VALUES ("a"), ("here") ON DUPLICATE KEY UPDATE `enable`=1;
COMMIT;
```

First, let's install the required modules if necessary

In [ ]:
!pip install redis pymysql pandas

# Initialise Database Connection

Here, we create a class for the `libs.database` submodule to initialise the connections.

The config is the same across all notebooks in this folder.
- The host is `mysql`
- The port that `mysql` runs on is `3306`
- The port that the `redis` service runs on is `6739`
- If you are testing locally with the database setup in singlish-words-backend, replace the username and password with your own MySQL username and password.

In [2]:
import json
cfg = json.load(open('config.json'))

class Configs:
    _db_host = cfg['_db_host']
    _db_port = cfg['_db_port']
    
    _db_name = cfg['_db_name']
    _db_charset = cfg['_db_charset']

    _redis_host = cfg['_redis_host']
    _redis_port = cfg['_redis_port']
    
    _db_user = cfg['_db_user']
    _db_passwd = cfg['_db_passwd']

This block will initialise the database connection.

In [4]:
import libs.database as database

database.initConnections(Configs)

## Read questions

It is required that the cues are in a `.csv` file separated by new lines! The block directly below can be changed to import the questions from any other location.

In [ ]:
question_list_file_name = "./export-data/cues.csv"

In [3]:
with open(question_list_file_name) as f:
    questions = f.read().splitlines()

## Refresh questions in database and cache

In [5]:
database.refreshMySQLQuestions(questions)

In [6]:
database.refreshRedis()

## Check if successfully inserted

This method check the current questions in the sample set.

In [ ]:
database.redisConnection.llen("questionList")